In [16]:
# !pip install 'zarr<3'
# !pip install timm
# !pip install openslide-python tiffslide
# !pip install cupy-cuda12x  # for CUDA 12.x

In [1]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## WSI Flex

In [2]:
import torch
from vitaminp import VitaminPFlex
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold2_best.pth", map_location=device))
model.eval()

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


VitaminPFlex(
  (backbone): DINOv2Backbone(
    (dinov2): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): M

## HE or mIF

In [6]:
from vitaminp.inference.channel_config import ChannelConfig
from vitaminp.inference import WSIPredictor

# Create channel config
config = ChannelConfig(
    nuclear_channel=0,  # SYTO 13
    membrane_channel=[1, 2],  # Combine all markers
    membrane_combination='max',
    channel_names={0: 'SYTO13', 1: 'Cy3', 2: 'TexasRed'}
)

# Create predictor with MIF config
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.4006,           # ← Default MPP (used if no metadata)
    magnification=20,          # ← Default magnification
    mif_channel_config=config
)

# Run MIF inference with MPP-aware filtering
results = predictor.predict(
    wsi_path='test_images/MS001S1_crop.ome.tiff',
    output_dir='results_new',
    branch='mif_cell',
    filter_tissue=False,
    # tissue_threshold=0.03,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,      # ← Binary threshold (0.5-0.8)
    min_area_um=5.0,              # ← NEW: Biological size filter in μm²
    # mpp_override=0.5,           # ← Optional: Force specific MPP
)

print(f"✅ Found {results['num_detections']} nuclei in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")

# Check if any instances have area info
if results['num_detections'] > 0:
    first_cell = list(results['instances'].values())[0]
    if 'area_um' in first_cell:
        print(f"   First cell area: {first_cell['area_um']:.2f} μm² ({first_cell['area_pixels']:.0f} pixels²)")

2026-01-18 16:56:01 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-18 16:56:01 - WSIPredictor - INFO -   Device: cuda
2026-01-18 16:56:01 - WSIPredictor - INFO -   Model type: VitaminPFlex (single-modality)
2026-01-18 16:56:01 - WSIPredictor - INFO -   Patch size: 512
2026-01-18 16:56:01 - WSIPredictor - INFO -   Overlap: 64
2026-01-18 16:56:01 - WSIPredictor - INFO -   Magnification: 20
2026-01-18 16:56:01 - WSIPredictor - INFO -   MIF channels: Nuclear: SYTO13, Membrane: max(Cy3, TexasRed)
2026-01-18 16:56:01 - WSIPredictor - INFO -    Using default MPP: 0.4006 μm/px
2026-01-18 16:56:01 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-18 16:56:01 - WSIPredictor - INFO -    WSI MPP: 0.4006 μm/px
2026-01-18 16:56:01 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-18 16:56:01 - WSIPredictor - INFO -    Scale factor: 1.52x
2026-01-18 16:56:01 - WSIPredictor - INFO -    Min area filter: 5.0 μm² = 31 pixels²
2026-01-18 16:56:01 - WSIPredictor - INFO - 📁

Output: shape=(2, 5000, 5000), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 7615x7615 (from 5000x5000)
   Scanning 17x17 tile grid...


2026-01-18 16:56:03 - WSIPredictor - INFO -    ✓ Created 289 tiles (17x17 grid)
2026-01-18 16:56:03 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on mif_cell...
Processing tiles: 100%|██████████| 289/289 [00:50<00:00,  5.68it/s]
2026-01-18 16:56:54 - WSIPredictor - INFO -    ✓ Extracted 30986 instances from tiles (before cleaning)
2026-01-18 16:56:54 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-18 16:56:54 - WSIPredictor - INFO -    Found 8998 edge cells to check for overlaps
2026-01-18 16:56:55 - WSIPredictor - INFO - Starting overlap cleaning with 30986 detections
2026-01-18 16:56:55 - WSIPredictor - INFO - Split detections: 0 mid, 30986 edge
2026-01-18 16:57:02 - WSIPredictor - INFO - Iteration 1: Found 9185 overlaps, keeping 21801/30986 detections
2026-01-18 16:57:05 - WSIPredictor - INFO - Iteration 2: Found 358 overlaps, keeping 21443/21801 detections
2026-01-18 16:57:09 - WSIPredictor - INFO - Iteration 3: Found 

✅ Found 21409 nuclei in 89.80s
   Output saved to: results_new


In [9]:
import tifffile
import numpy as np

input_path = '/rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/DSP Scans/MS001S1.ome.tiff'

with tifffile.TiffFile(input_path) as tif:
    print("="*60)
    print("BASIC IMAGE INFO")
    print("="*60)
    
    image = tif.asarray()
    print(f"Shape: {image.shape}")
    print(f"Dtype: {image.dtype}")
    print(f"Size in memory: {image.nbytes / (1024**3):.2f} GB")
    print(f"Number of pages/series: {len(tif.pages)}")
    print(f"Is OME-TIFF: {tif.is_ome}")
    
    print("\n" + "="*60)
    print("VALUE RANGES PER CHANNEL")
    print("="*60)
    
    for i in range(image.shape[0]):
        channel_data = image[i]
        print(f"Channel {i}:")
        print(f"  Min: {channel_data.min()}")
        print(f"  Max: {channel_data.max()}")
        print(f"  Mean: {channel_data.mean():.2f}")
        print(f"  Non-zero pixels: {np.count_nonzero(channel_data)} ({100*np.count_nonzero(channel_data)/channel_data.size:.2f}%)")
    
    print("\n" + "="*60)
    print("DATA LOCATION (NON-ZERO REGIONS)")
    print("="*60)
    
    # Check each channel for where data exists
    for i in range(image.shape[0]):
        nonzero = np.nonzero(image[i] > 0)
        if len(nonzero[0]) > 0:
            y_min, y_max = nonzero[0].min(), nonzero[0].max()
            x_min, x_max = nonzero[1].min(), nonzero[1].max()
            print(f"Channel {i} - Data region:")
            print(f"  Y: {y_min} to {y_max} (height: {y_max-y_min})")
            print(f"  X: {x_min} to {x_max} (width: {x_max-x_min})")
        else:
            print(f"Channel {i} - No data (all zeros)")
    
    print("\n" + "="*60)
    print("TIFF TAGS (First Page)")
    print("="*60)
    
    page = tif.pages[0]
    print(f"Image width: {page.imagewidth}")
    print(f"Image height: {page.imagelength}")
    print(f"Bits per sample: {page.bitspersample}")
    print(f"Compression: {page.compression}")
    print(f"Photometric: {page.photometric}")
    print(f"Samples per pixel: {page.samplesperpixel}")
    
    print("\n" + "="*60)
    print("OME METADATA (first 1000 chars)")
    print("="*60)
    
    if tif.ome_metadata:
        print(tif.ome_metadata[:1000])

BASIC IMAGE INFO
Shape: (4, 32768, 32768)
Dtype: uint16
Size in memory: 8.00 GB
Number of pages/series: 4
Is OME-TIFF: True

VALUE RANGES PER CHANNEL
Channel 0:
  Min: 0
  Max: 65535
  Mean: 1616.40
  Non-zero pixels: 655847059 (61.08%)
Channel 1:
  Min: 0
  Max: 65535
  Mean: 1023.67
  Non-zero pixels: 662886473 (61.74%)
Channel 2:
  Min: 0
  Max: 35504
  Mean: 201.73
  Non-zero pixels: 663340525 (61.78%)
Channel 3:
  Min: 0
  Max: 48175
  Mean: 394.93
  Non-zero pixels: 663710143 (61.81%)

DATA LOCATION (NON-ZERO REGIONS)
Channel 0 - Data region:
  Y: 2351 to 30207 (height: 27856)
  X: 3209 to 29439 (width: 26230)
Channel 1 - Data region:
  Y: 2351 to 30207 (height: 27856)
  X: 3209 to 29439 (width: 26230)
Channel 2 - Data region:
  Y: 2351 to 30207 (height: 27856)
  X: 3209 to 29439 (width: 26230)
Channel 3 - Data region:
  Y: 2351 to 30207 (height: 27856)
  X: 3209 to 29439 (width: 26230)

TIFF TAGS (First Page)
Image width: 32768
Image height: 32768
Bits per sample: 16
Compression

In [10]:
import tifffile

# Input and output paths
input_path = '/rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/DSP Scans/MS001S1.ome.tiff'
output_path = '/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/MS001S1_crop.ome.tiff'

# Crop parameters - cropping from the center of the data region
# Data region is Y: 2351-30207, X: 3209-29439
# Center would be around Y: 16279, X: 16324
crop_size = 5000
x_start = 14000  # Center region with data
y_start = 14000  # Center region with data

print(f"Cropping region: Y[{y_start}:{y_start+crop_size}], X[{x_start}:{x_start+crop_size}]")

# Read the OME-TIFF file
with tifffile.TiffFile(input_path) as tif:
    image = tif.asarray()
    
    print(f"Original image shape: {image.shape}")
    print(f"Original dtype: {image.dtype}")
    
    # Crop the image - preserve all 4 channels
    cropped = image[:, y_start:y_start+crop_size, x_start:x_start+crop_size]
    
    print(f"Cropped image shape: {cropped.shape}")
    print(f"\nCropped value ranges:")
    for i in range(cropped.shape[0]):
        print(f"  Channel {i}: min={cropped[i].min()}, max={cropped[i].max()}, mean={cropped[i].mean():.2f}")
    
    # Get compression from original
    compression = tif.pages[0].compression

print(f"\nSaving with compression type: {compression}")

# Save as OME-TIFF preserving the format
with tifffile.TiffWriter(output_path, bigtiff=True) as tiff_out:
    tiff_out.write(
        cropped,
        photometric='minisblack',
        compression=compression,
        metadata={'axes': 'CYX'},
    )

print(f"\nCropped image saved to: {output_path}")

Cropping region: Y[14000:19000], X[14000:19000]
Original image shape: (4, 32768, 32768)
Original dtype: uint16
Cropped image shape: (4, 5000, 5000)

Cropped value ranges:
  Channel 0: min=0, max=59797, mean=2903.80
  Channel 1: min=0, max=65535, mean=2367.08
  Channel 2: min=0, max=14550, mean=340.70
  Channel 3: min=0, max=37068, mean=696.13

Saving with compression type: 5

Cropped image saved to: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/MS001S1_crop.ome.tiff


In [13]:
import tifffile
import numpy as np

def check_ome_tiff_info(image_path, name):
    print("\n" + "="*70)
    print(f"CHECKING: {name}")
    print("="*70)
    
    with tifffile.TiffFile(image_path) as tif:
        image = tif.asarray()
        
        print(f"\n📐 DIMENSIONS:")
        print(f"   Shape: {image.shape}")
        print(f"   Dtype: {image.dtype}")
        print(f"   Is OME: {tif.is_ome}")
        
        print(f"\n🔍 CHANNEL INFO:")
        for i in range(min(4, image.shape[0] if len(image.shape) > 2 else 1)):
            if len(image.shape) == 2:
                ch = image
            elif len(image.shape) == 3 and image.shape[2] <= 4:
                ch = image[:, :, i]
            else:
                ch = image[i]
            print(f"   Channel {i}: min={ch.min()}, max={ch.max()}, mean={ch.mean():.2f}, non-zero={100*np.count_nonzero(ch)/ch.size:.1f}%")
        
        print(f"\n📏 METADATA:")
        page = tif.pages[0]
        print(f"   Image size: {page.imagewidth} x {page.imagelength}")
        print(f"   Bits per sample: {page.bitspersample}")
        print(f"   Compression: {page.compression}")
        print(f"   Samples per pixel: {page.samplesperpixel}")
        
        # Check for resolution/MPP info
        if hasattr(page, 'tags'):
            if 'XResolution' in page.tags:
                x_res = page.tags['XResolution'].value
                print(f"   XResolution: {x_res}")
            if 'YResolution' in page.tags:
                y_res = page.tags['YResolution'].value
                print(f"   YResolution: {y_res}")
            if 'ResolutionUnit' in page.tags:
                res_unit = page.tags['ResolutionUnit'].value
                print(f"   ResolutionUnit: {res_unit}")
        
        # Check OME metadata for PhysicalSize
        if tif.ome_metadata:
            ome_str = tif.ome_metadata
            if 'PhysicalSizeX' in ome_str:
                import re
                phys_x = re.search(r'PhysicalSizeX="([^"]+)"', ome_str)
                phys_y = re.search(r'PhysicalSizeY="([^"]+)"', ome_str)
                phys_unit = re.search(r'PhysicalSizeXUnit="([^"]+)"', ome_str)
                if phys_x and phys_y:
                    print(f"   PhysicalSizeX: {phys_x.group(1)} {phys_unit.group(1) if phys_unit else ''}")
                    print(f"   PhysicalSizeY: {phys_y.group(1)} {phys_unit.group(1) if phys_unit else ''}")
                    
                    # Calculate MPP if in micrometers
                    if phys_unit and 'µm' in phys_unit.group(1):
                        mpp = float(phys_x.group(1))
                        print(f"   ⚠️ CALCULATED MPP: {mpp} μm/pixel")
            else:
                print(f"   ⚠️ No PhysicalSize metadata found")
        
        print(f"\n📄 OME METADATA (relevant parts):")
        if tif.ome_metadata:
            # Print lines containing resolution/size info
            for line in tif.ome_metadata.split('\n')[:30]:
                if any(keyword in line for keyword in ['PhysicalSize', 'Pixels', 'DimensionOrder']):
                    print(f"   {line.strip()}")

# Check both images
check_ome_tiff_info('test_images/MS001S1_crop.ome.tiff', 'MS001S1_crop (PROBLEMATIC)')
check_ome_tiff_info('test_images/mif_region.ome.tiff', 'mif_region (WORKING)')


CHECKING: MS001S1_crop (PROBLEMATIC)

📐 DIMENSIONS:
   Shape: (4, 5000, 5000)
   Dtype: uint16
   Is OME: True

🔍 CHANNEL INFO:
   Channel 0: min=0, max=59797, mean=2903.80, non-zero=100.0%
   Channel 1: min=0, max=65535, mean=2367.08, non-zero=100.0%
   Channel 2: min=0, max=14550, mean=340.70, non-zero=100.0%
   Channel 3: min=0, max=37068, mean=696.13, non-zero=100.0%

📏 METADATA:
   Image size: 5000 x 5000
   Bits per sample: 16
   Compression: 5
   Samples per pixel: 1
   XResolution: (1, 1)
   YResolution: (1, 1)
   ResolutionUnit: 1
   ⚠️ No PhysicalSize metadata found

📄 OME METADATA (relevant parts):
   <?xml version="1.0" encoding="UTF-8"?><OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:807c752a-f486-11f0-8756-ea0ee0401239"  Creator="tifffile.py 2023.12.9"><Image ID

## Dual model

In [ ]:
import torch
from vitaminp import VitaminPDual
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPDual(model_size='base').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_dual_base_fold1_best.pth", map_location=device))
model.eval()

Building H&E encoder with DINOv2-base


In [24]:

# Corrected channel config for MIF
config = ChannelConfig(
    nuclear_channel=2,      # DAPI is in channel 2 (Blue)
    membrane_channel=[0, 1], # Cell markers in channels 0 and 1 (Red + Green)
    membrane_combination='max',
    channel_names={0: 'CellMarker1', 1: 'CellMarker2', 2: 'DAPI'}
)

# Default behavior (H&E predictions for H&E)
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.263,
    magnification=40,
    mif_channel_config=config
)

# 🔥 NEW: Use MIF predictions for H&E (seamlessly)
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.263,
    magnification=40,
    mif_channel_config=config,
)

# Same predict call - no changes needed!
results = predictor.predict(
    wsi_path='test_images/prostate-he_chunk_12.png',
    wsi_path_mif='test_images/prostate-mif_chunk_12_combined.png',
    output_dir='results_dual_new',
    branches=['he_nuclei', 'he_cell', 'mif_nuclei', 'mif_cell'],
    filter_tissue=True,
    clean_overlaps=True,
    save_geojson=True,
    save_visualization=True,
    detection_threshold=0.5,
    min_area_um=3.0,
)

# H&E results now contain MIF predictions automatically!
print(f"✅ H&E nuclei: {results['he_nuclei']['num_detections']}")
print(f"✅ H&E cells: {results['he_cell']['num_detections']}")

2026-01-17 22:44:06 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-17 22:44:06 - WSIPredictor - INFO -   Device: cuda
2026-01-17 22:44:06 - WSIPredictor - INFO -   Model type: VitaminPDual (dual-modality)
2026-01-17 22:44:06 - WSIPredictor - INFO -   Patch size: 512
2026-01-17 22:44:06 - WSIPredictor - INFO -   Overlap: 64
2026-01-17 22:44:06 - WSIPredictor - INFO -   Magnification: 40
2026-01-17 22:44:06 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: max(CellMarker1, CellMarker2)
2026-01-17 22:44:06 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-17 22:44:06 - WSIPredictor - INFO -   Device: cuda
2026-01-17 22:44:06 - WSIPredictor - INFO -   Model type: VitaminPDual (dual-modality)
2026-01-17 22:44:06 - WSIPredictor - INFO -   Patch size: 512
2026-01-17 22:44:06 - WSIPredictor - INFO -   Overlap: 64
2026-01-17 22:44:06 - WSIPredictor - INFO -   Magnification: 40
2026-01-17 22:44:06 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: 

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ Extracted 91 instances from tiles (before cleaning)
2026-01-17 22:44:07 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:07 - WSIPredictor - INFO -    Found 33 edge cells to check for overlaps
2026-01-17 22:44:07 - WSIPredictor - INFO - Starting overlap cleaning with 91 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - Split detections: 0 mid, 91 edge
2026-01-17 22:44:07 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 91/91 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:07 - WSIPredictor - INFO - Cleaning complete: 91 -> 91 (0 removed)
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ After cleaning: 91 instances
2026-01-17 22:44:07 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/he_nuclei...
2026-01-17 22:44:07 - WSIPredictor - INFO 

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  4.49it/s]
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ Extracted 89 instances from tiles (before cleaning)
2026-01-17 22:44:07 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:07 - WSIPredictor - INFO -    Found 34 edge cells to check for overlaps
2026-01-17 22:44:07 - WSIPredictor - INFO - Starting overlap cleaning with 89 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - Split detections: 0 mid, 89 edge
2026-01-17 22:44:07 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 89/89 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:07 - WSIPredictor - INFO - Cleaning complete: 89 -> 89 (0 removed)
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ After cleaning: 89 instances
2026-01-17 22:44:07 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/he_cell...
2026-01-17 22:44:07 - WSIPredictor - INFO - 

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  4.74it/s]
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ Extracted 91 instances from tiles (before cleaning)
2026-01-17 22:44:08 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:08 - WSIPredictor - INFO -    Found 33 edge cells to check for overlaps
2026-01-17 22:44:08 - WSIPredictor - INFO - Starting overlap cleaning with 91 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - Split detections: 0 mid, 91 edge
2026-01-17 22:44:08 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 91/91 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:08 - WSIPredictor - INFO - Cleaning complete: 91 -> 91 (0 removed)
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ After cleaning: 91 instances
2026-01-17 22:44:08 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/mif_nuclei...
2026-01-17 22:44:08 - WSIPredictor - INFO

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  3.96it/s]
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ Extracted 89 instances from tiles (before cleaning)
2026-01-17 22:44:08 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:08 - WSIPredictor - INFO -    Found 34 edge cells to check for overlaps
2026-01-17 22:44:08 - WSIPredictor - INFO - Starting overlap cleaning with 89 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - Split detections: 0 mid, 89 edge
2026-01-17 22:44:08 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 89/89 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:08 - WSIPredictor - INFO - Cleaning complete: 89 -> 89 (0 removed)
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ After cleaning: 89 instances
2026-01-17 22:44:08 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/mif_cell...
2026-01-17 22:44:08 - WSIPredictor - INFO -

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
✅ H&E nuclei: 91
✅ H&E cells: 89


## DSP set 1

In [5]:
import tifffile
import numpy as np

# Input and output paths
input_path = '/rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/DSP Scans/MS001S1.ome.tiff'
output_path = '/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/MS001S1_crop.ome.tiff'

# Crop parameters
crop_size = 5000
x_start = 0  # adjust these coordinates as needed
y_start = 0

# Read the OME-TIFF file
with tifffile.TiffFile(input_path) as tif:
    # Get the image data
    image = tif.asarray()
    
    # Get OME metadata to preserve it
    ome_metadata = tif.ome_metadata
    
    print(f"Original image shape: {image.shape}")
    
    # Crop the image - shape is (4, 32768, 32768)
    cropped = image[:, y_start:y_start+crop_size, x_start:x_start+crop_size]
    
    print(f"Cropped image shape: {cropped.shape}")

# Save as OME-TIFF with correct axes
# The shape (4, 5000, 5000) could be:
# - CYX (4 channels) or
# - ZYX (4 Z-slices)
# Most likely it's channels, so using 'CYX'
tifffile.imwrite(
    output_path,
    cropped,
    photometric='minisblack',
    metadata={'axes': 'CYX'},  # Changed to match 3D shape
)

print(f"Cropped image saved to: {output_path}")

Original image shape: (4, 32768, 32768)
Cropped image shape: (4, 5000, 5000)
Cropped image saved to: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/MS001S1_crop.ome.tiff
